In [2]:
import pandas as pd
import datetime
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score, recall_score, f1_score,ConfusionMatrixDisplay, confusion_matrix
import numpy as np

In [3]:
cri_compustat_merged = pd.read_pickle("cri_compustat_merged_v8.pkl")
cri_compustat_merged.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cri_compustat_merged_v8.pkl'

In [ ]:
us = pd.read_csv("US.csv", skiprows=3)
us.rename(columns={'Company_number':'CompNo'}, inplace=True)
us = us.merge(cri_compustat_merged, on=['CompNo','year','month'], how='inner')

#refactor
cols = list(us.columns)

us.sort_values(by='EventDate', inplace=True)
date2022 = datetime.datetime(2022, 1, 1)
date2020 = datetime.datetime(2020,1,1)
train_df_us = us[us["EventDate"] < date2020]
validation_df_us = us[(us["EventDate"] >= date2020) & (us["EventDate"] < date2022)]
test_df_us = us[us["EventDate"] >= date2022]

train_df_us.drop(["CompNo", "year", "month"], axis=1, inplace=True)
validation_df_us.drop(["CompNo", "year", "month"], axis=1, inplace=True)
test_df_us.drop(["CompNo", "year", "month"], axis=1, inplace=True)

In [ ]:
x_train_us = train_df_us.drop('Default', axis=1)
y_train_us = train_df_us['Default']
x_validation_us = validation_df_us.drop('Default', axis=1)
y_validation_us = validation_df_us['Default']
x_test_us = test_df_us.drop('Default', axis=1)
y_test_us = test_df_us["Default"]

In [ ]:
y_pred_us = x_test_us['12_month']
y_pred_binary_us = np.round(y_pred_us)

In [ ]:
accuracy = accuracy_score(y_test_us, y_pred_binary_us)
recall = recall_score(y_test_us, y_pred_binary_us)
precision = precision_score(y_test_us, y_pred_binary_us)
f1 = f1_score(y_test_us, y_pred_binary_us)
auc = roc_auc_score(y_test_us, y_pred_us)
confusion = confusion_matrix(y_test_us, y_pred_binary_us)

print('Accuracy:', accuracy)
print('Recall:', recall)
print('Precision:', precision)
print('F1 Score:', f1)
print('AUC:', auc)


ConfusionMatrixDisplay(confusion).plot()

In [ ]:
us.isna().sum()